In [1]:
%defaultDatasource jdbc:h2:mem:db

# Análise de taxas de vacinação

In [2]:
DROP VIEW IF EXISTS Medias_Vacinas;
DROP VIEW IF EXISTS Medias_Antivax;

DROP TABLE IF EXISTS Antivax_Trends;
DROP TABLE IF EXISTS Immunization_Estimates;

CREATE TABLE Antivax_Trends (
    Periodo INTEGER,
    Popularidade INTEGER,
    PRIMARY KEY(Periodo)
) AS SELECT
    Periodo,
    Popularidade
FROM CSVREAD('../data/vacinas/trends-antivax-anual.csv');

CREATE TABLE Immunization_Estimates (
    Id INTEGER NOT NULL,
    vaccine VARCHAR,
    Ano INTEGER,
    Taxa INTEGER,
    PRIMARY KEY(Id)
)AS SELECT
    Id,
    vaccine,
    Ano,
    Taxa
FROM CSVREAD('../data/vacinas/Immunization-estimates-2018-Brasil.csv')


In [3]:
SELECT * FROM Antivax_Trends;
SELECT * FROM Immunization_Estimates;

# I. Análise de comportamento estatístico

## 1) Estruturação de view para assitência na análise

In [4]:
CREATE VIEW IF NOT EXISTS Medias_Vacinas AS 
    SELECT vaccine, AVG(Immunization_Estimates.Taxa) Media_taxa
    FROM Immunization_Estimates
    GROUP BY vaccine;
    
SELECT * FROM Medias_Vacinas;

## 2) Para cada ano a quantidade de taxas de imunização acima da média por vacina

In [13]:
DROP VIEW IF EXISTS contador_acima_media;
CREATE VIEW contador_acima_media AS 
    Select Ano Anos_Acima_Da_Media, Immunization_Estimates.vaccine, Immunization_Estimates.taxa, Immunization_Estimates.ano
    From Medias_Vacinas, Immunization_Estimates
    Where Immunization_Estimates.Taxa >= Medias_Vacinas.media_taxa AND Immunization_Estimates.ano = Ano AND Medias_Vacinas.vaccine = Immunization_Estimates.vaccine
    Group by ano, Immunization_Estimates.vaccine;
    
    --para cada ano eu quero selecionar a contagem de valores acima da media

SELECT ano Ano_acima_media, COUNT(*) 
    FROM contador_acima_media
    GROUP BY ano;

SELECT * FROM contador_acima_media;

## 3) Para cada ano a quantidade de taxas de imunização abaixo da média por vacina

In [11]:
DROP VIEW IF EXISTS contador_abaixo_media;
CREATE VIEW contador_abaixo_media AS 
    Select Ano Anos_Abaixo_Da_Media, Immunization_Estimates.vaccine, Immunization_Estimates.taxa, Immunization_Estimates.ano
    From Medias_Vacinas, Immunization_Estimates
    Where Immunization_Estimates.Taxa < Medias_Vacinas.media_taxa AND Immunization_Estimates.ano = Ano AND Medias_Vacinas.vaccine = Immunization_Estimates.vaccine
    Group by ano, Immunization_Estimates.vaccine;
    
    --para cada ano eu quero selecionar a contagem de valores abaixo da media

SELECT ano Ano_abaixo_media, COUNT(*) 
    FROM contador_abaixo_media
    GROUP BY ano;
    
SELECT * FROM contador_abaixo_media;

## 4) Período com maior número de pesquisas contra vacinação

In [6]:
Select Periodo 
    from Antivax_Trends
    WHERE Antivax_Trends.Popularidade = (Select MAX(Popularidade) 
                                             from Antivax_Trends);

2018